<h1 style="color:#00A6D6;">Introduction to Quantum Cryptography - Jupyter Notebooks</h1>
<h2 style="color:#00A6D6;">Chapter 3: Quantum Money</h2>

Welcome to the new Julia sheet! As usual, we will ask you to use Julia to answer a few exercises. Most importantly, however, the purpose of these Julia sheets is for you to play around and build intuition by exploring and calculating things we do NOT ask you :-) We hope that you take advantage of using Julia this way.

In this Julia Jupyter notebook, we focus on exploring Wiesner's quantum money scheme in more detail!

* <a href="#wiesner"> Wiesner's Quantum Money </a>

The include command that follows will include all the functions that you have used on the previous chapters.

In [ ]:
include("source/main.jl");

<a id="bb84"></a>
<h2 style="color:#00A6D6;">Wiesner's quantum money</h2>

In the below we will investigate some features of Wiesner's quantum money scheme using very tiny quantum coins. As you may recall from the book, in this we would like to implement both an ability to generate using a procedure GEN, supplying us with a serial number, a quantum coin state, as well as the corresponding key. The bank will use GEN, and pass the serial number and the quantum coin state to the coin carrier. The bank is able to verify coins using a verification procedure VER that checks whether a supplied coin state with a specific serial number matches the key stored. For Wiesner's scheme this looks more specifically like this:

<ul>
    <li>The bank executes $GEN(1^n)$ producing a serial number $S$, a quantum coin state $|\Psi_S\rangle$, and key $k$. Here, $k = (x,\theta)$, where $x \in \{0,1\}^n$ and $\theta \in \{0,1\}^n$ are generated uniformly at random. The coin state is prepared as $$|\Psi_S\rangle = \bigotimes_{i=0}^{n-1} H^{\theta_i} |x_i\rangle\ , $$ where $H$ is the Hadamard matrix, and $\{|0\rangle,|1\rangle\}$ the standard basis. The bank keeps $(S,k)$, and hands $|\Psi_S\rangle$ to a customer.</li>
    <li> The bank allows a verification service $VER$ that can be used to verify coins. It takes as input a serial number $S$, and a coin state $\sigma$. Given $S$, the bank looks up the matching key $k$, and measures qubit $i$ of $\sigma$ in the basis given by $\theta_i$ to obtain outcome $\tilde{x}_i$. If $\tilde{x}_i = x_i$ for all $i$, the the bank accepts. Otherwise the bank rejects the verification.
</ul>

In [ ]:
# Let's define the elements of the BB84 Basis as ket vectors
# The first index indicates the basis and the second the basis element
v0 = [1 0]';
v1 = [0 1]';
w0 = ([1 1]')/sqrt(2);
w1 = ([1 -1]')/sqrt(2);

# The BB84 states. Note that due to Julia indexing from 1 onwards, 1 is the standard basis, and 2 the Hadamard one
# Similarly bit x=0 will correspond to index 1, and x=1 to index 2
BB84 = [[v0] [v1]; [w0] [w1]];

# Let's set how long a coin Alice and Bob will produce. 
n = 50;

# For this example, we will just use a fixed serial number
S = 1;

# Strings to be produced
x = zeros(Int8,n); 
theta = zeros(Int8,n);
coin = Dict();

# Generate the coin, we will keep is separated by qubit where i will index the qubit
for i = 1:n

        # Alice picks a random BB84 state
        # Bit of the string
        x[i] = rand(0:1);
        # Basis to encode in
        theta[i] = rand(0:1);
    
        # Here Alice would send the state to Bob
        coin[i] = BB84[theta[i]+1,x[i]+1][1:2];

end

# print("Generated quantum money with serial number ", S, " key (", x, ",", theta, ") and coin state: ", coin, "\n");


<h3 style="color:#00A6D6;"> Exercise 1</h3>
Let's verify some coins! Use the code above to produce some - randomly generated coins - and verify them! Did you expect to see any wrong outcomes?

In [ ]:
# Verify the coin, since we set S to be a constant in this example, we will just ignore S

# Use the key k=(x,theta) to determine what should be the correct coin state
wrongOutcomes = 0;
for i = 1:n
        # Check whether the i-th coin matches

        # Compute the probability of getting the correct outcome
        p0 = round((((BB84[theta[i]+1,x[i]+1][1:2])' * coin[i]))^2; digits=10);

        # and the wrong outcome
        p1 = 1-p0;

        # Sample from {0,1} where p1 is given to simulate the measurement 
        rndDist = Binomial(1,p1);
        outcome = rand(rndDist);

        if outcome == 1
            # wrong outcome
            wrongOutcomes = wrongOutcomes + 1;
        end
    
end

print("Error rate: ", wrongOutcomes/n, "\n");


<h3 style="color:#00A6D6;"> Exercise 2</h3>

Let's now imagine that we have an honest user of the bank, but his quantum memory is not perfect. Specifically, imagine that while in memory, the quantum coin undergoes noise described by one of the following two types of noise:
<ul> 
    <li>With probability $q$, a phase flip $Z$ is applied to each qubit.</li>
    <li> With probability $q$, a bit flip $X$ is applied to each qubit.</li>
   </ul>
Adapt the code above to investigate what happens in each of the two cases. Examine the relationship between the probability $q$, and the error rate in the verification procedure. 

In [ ]:
# Below you can find a piece of example code that applies random noise to a specific BB84 state.
# Adapt the code below to apply errors to the qubits of the coin, and run the verification procedure on the noisy coin. 
# You may wish to copy paste the verification code above below

# phase flip Z
Z = [1 0;0 -1];

# bit flip X
X = [0 1;1 0];

# Example: Let's see how we could simulate a noisy channel that applies an error matrix with probability q to an example state
Psi = BB84[1,1][1:2];
errPsi = Psi;
q = 0.9 # error probability
errorRnd = Bernoulli(q);
error = rand(errorRnd);
if error
    errPsi = X * Psi;
end
print("Input state was ",Psi," output state is ", errPsi, "\n");

<h3 style="color:#00A6D6;">Exercise 3</h3>
In the case that the coins undergo noise, when would the verification procedure of the bank accept? It is intuitively clear that in this case, the bank might need to allow some small error rate $\epsilon$ of errors and still accept the verification as being correct. This would allow a quantum money scheme that is tolerant to some amount of noise in the quantum devices used to store or communicate the coins. Think about ways to adapt the verification procedure, where you may wish to experiment using Julia below.

In [ ]:
# Your code goes here

<h3 style="color:#00A6D6;"> Exercise 4</h3>
Your goal in this exercise will be to try and cheat!
<ul>
    <li>Imagine that you have never interacted with the bank before. Your goal is to come up with a state $\sigma = \rho^{\otimes n}$ (for simplicity) for which the bank may (depending on its tolerance to errors above!) still want to accept. That is your goal is to find a state $\rho$ that makes the error rate in verification low. Test your state by adapting the code above. Note that the code above assumes the coin is a tensor product of states (to avoid using too much memory in these examples!) and for each qubit i, the coin coin[i] is given as a ket vector. </li>
    <li> For $n=1$, investigate the cloning attack suggested in Chapter 3.5.1. </li>
</ul>


In [ ]:
# Your code goes here. Note that his requires executing the code above, which also sets n.
# you may wish to experiment with different values of n above

# As an example, we use sigma to be a cheating coin where each qubit is simply rho = |0><0|
cheatingCoin = Dict()
for i = 1:n
        cheatingCoin[i] = [1 ; 0]; # Adapt this
end

# Run the verification procedure given above using the cheatingCoin
wrongOutcomes = 0;
for i = 1:n
        # Check whether the i-th coin matches

        # Compute the probability of getting the correct outcome
        p0 = round((((BB84[theta[i]+1,x[i]+1][1:2])' * cheatingCoin[i]))^2; digits=10);

        # and the wrong outcome
        p1 = 1-p0;

        # Sample from {0,1} where p1 is given to simulate the measurement 
        rndDist = Binomial(1,p1);
        outcome = rand(rndDist);

        if outcome == 1
            # wrong outcome
            wrongOutcomes = wrongOutcomes + 1;
        end
    
end

print("Error rate: ", wrongOutcomes/n, "\n");